In [1]:
import numpy as np
import os
import shutil
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy
import time
import pickle

# Read MLI input file

In [2]:
elems0,lattices,labor = mli.readInputfile('mli.in.t3_iota_8_4_t0p4')
del(lattices[-1])

setarclen is not recognized. skipping...
autoconcat is not recognized. skipping...
autoapply is not recognized. skipping...
reftraj is not recognized. skipping...
fit is not recognized. skipping...
bip is not recognized. skipping...
tip is not recognized. skipping...


In [3]:
energy=elems0[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [4]:
elems = []
for item in elems0:
    if not item.name in ['tasm','aim','vary','clear','anaprint']:
        elems.append(item)
elems[2].driftexact = 1

In [5]:
labor = ['iotaline','mapout','fin']

# thick sext 2 thin multipole

In [6]:
elemList,latticeList=mli.sext2thin(elems,lattices,brho=0.50204778184582999)

In [7]:
mli.writeInputfile(elemList,latticeList,labor)

In [8]:
mli.run()

In [9]:
M,G=mli.readTransferMap()

In [10]:
M

,1,2,3,4,5,6
1,0.999968,0.000070,0.000000,0.000000e+00,0.0,-1.147949e-06
2,-1.454445,0.999930,0.000000,0.000000e+00,0.0,-2.587771e-07
3,0.000000,0.000000,0.999995,3.984744e-07,0.0,0.000000e+00
4,0.000000,0.000000,-1.454438,1.000004e+00,0.0,0.000000e+00
5,0.000002,-0.000001,0.000000,0.000000e+00,1.0,-1.297155e+00
6,0.000000,0.000000,0.000000,0.000000e+00,0.0,1.000000e+00


In [11]:
G.head()

,exponents,GP
28,f( 30 00 00 ),-23.555531
29,f( 21 00 00 ),-62.881728
33,f( 20 00 01 ),21.191487
34,f( 12 00 00 ),-51.643332
38,f( 11 00 01 ),54.729188


In [12]:
indexThin = []
k2lList = []
for i,item in enumerate(elemList):
    if 'thlm' == item.elem:
        indexThin.append(i)
        k2lList.append(item.k2l)
nThin = len(indexThin)
print(nThin)

18


# Optimize

### Build objective

In [13]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx
print(k)

1.4544633270832747


In [14]:
emit = 3.3e-6
bg = gam0*bet0
sx = 2*np.sqrt(betx*emit)
spx = 2*np.sqrt((1+alfx*alfx)/betx*emit)
sy = 2*np.sqrt(betx*emit)
spy = 2*np.sqrt((1+alfx*alfx)/betx*emit)
se = 2*2.0e-3*bet0
print(sx,spx,se)

0.00499827394686418 0.004492987295060738 0.0039999769466163605


In [15]:
def getWeight(g):
    w = np.zeros(len(g))
    for i,item in enumerate(g['exponents']):
        nx = int(item[3])
        npx = int(item[4])
        ny = int(item[6])
        npy = int(item[7])
        nE = int(item[10])
        w[i] = np.power(sx,nx)*np.power(spx,npx)*np.power(sy,ny)*np.power(spy,npy)*np.power(se,nE)
    return w

In [16]:
G2_ref = G.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0
W2 = getWeight(G2_ref)
G2_ref

,exponents,GP
28,f( 30 00 00 ),0.000000
29,f( 21 00 00 ),0.000000
33,f( 20 00 01 ),0.727236
34,f( 12 00 00 ),0.000000
38,f( 11 00 01 ),0.000000
39,f( 10 20 00 ),0.000000
40,f( 10 11 00 ),0.000000
43,f( 10 02 00 ),0.000000
48,f( 10 00 02 ),0.000000
49,f( 03 00 00 ),0.000000


In [17]:
G3_ref = G.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2
W3 = getWeight(G3_ref)
G3_ref

,exponents,GP
84,f( 40 00 00 ),0.000000
85,f( 31 00 00 ),0.000000
89,f( 30 00 01 ),0.000000
90,f( 22 00 00 ),0.000000
94,f( 21 00 01 ),0.000000
95,f( 20 20 00 ),0.000000
96,f( 20 11 00 ),0.000000
99,f( 20 02 00 ),0.000000
104,f( 20 00 02 ),0.000004
105,f( 13 00 00 ),0.000000


In [18]:
G4_ref = G.loc[210:450].copy()
G4_ref['GP'] = 0
W4 = getWeight(G4_ref)

In [19]:
G5_ref = G.loc[462:910].copy()
G5_ref['GP'] = 0
W5 = getWeight(G5_ref)

### Obj

In [20]:
#%%
def objFunc(arg): 
    target = opt.id_generator()  # generage random directory name
    while os.path.exists(target):  
        target = opt.id_generator()
    shutil.copytree('origin', target)
    os.chdir(target) # cd to the randome directory and
    
    for i,j in enumerate(indexThin):
        elemList[j]['k2l']=arg[i]
    for i,j in enumerate(indexThin):
        elemList[j]['k3l']=arg[i+nThin]
        
    mli.writeInputfile(elemList,latticeList,labor)
    mli.run()
    M,G = mli.readTransferMap()
    
    test = np.all(G2_ref['exponents'].values == G.loc[28 :76 ]['exponents'].values) and \
           np.all(G3_ref['exponents'].values == G.loc[84 :200]['exponents'].values) and \
           np.all(G4_ref['exponents'].values == G.loc[210:450]['exponents'].values) and \
           np.all(G5_ref['exponents'].values == G.loc[462:910]['exponents'].values)
    
    if not test:
        raise ValueError("PROBLEM!!!")
        
    obj = np.sum(((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)*W2)**2) + \
          np.sum(((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)*W3)**2) + \
          np.sum( (G.loc[210:450,'GP'].values*W4)**2) + \
          np.sum( (G.loc[462:910,'GP'].values*W5)**2)
    
    os.chdir('..')
    shutil.rmtree(target)
    return obj

In [21]:
objFunc([0]*2*nThin)

1.4389907528541867e-10

In [22]:
# #%% run optim
bounds = [(-50,50)]*nThin + [(-500,500)]*nThin
# result=opt.differential_evolution(objFunc, bounds, ncore=32, popsize=32*8,
#                                   disp=True, polish=True, maxtime=60*10) 
#                                     stop running at maximum 10 min

In [23]:
# with open('result.thin.2sig.sext.oct','wb') as fp:
#     pickle.dump(result,fp)

In [24]:
with open('result.thin.2sig.sext.oct','rb') as fp:
    result=pickle.load(fp)

In [25]:
result.message

'Maximum time has been exceeded.'

In [26]:
np.sort(result.population_energies)[:16]

array([4.17980963e-11, 4.17980963e-11, 4.38148041e-11, 4.38252344e-11,
       4.43744455e-11, 4.54827950e-11, 4.67602410e-11, 4.69049298e-11,
       4.69584905e-11, 4.77186211e-11, 4.77577399e-11, 4.81665180e-11,
       4.88080430e-11, 4.88647614e-11, 4.91724145e-11, 4.99037840e-11])

In [27]:
np.sort(result.population_energies)[-16:]

array([8.20154784e-11, 8.23858440e-11, 8.30685761e-11, 8.34984957e-11,
       8.40150329e-11, 8.53228682e-11, 8.60673228e-11, 8.85304881e-11,
       8.86611965e-11, 8.90583790e-11, 8.92003990e-11, 9.12789227e-11,
       9.15037957e-11, 9.28803897e-11, 9.54045792e-11, 9.69070976e-11])

In [28]:
# pop = copy(result.population[0]-0.5)
# pop[:nThin] = pop[:nThin]*200
# pop

In [ ]:
while True:
    previous_result = result
    if hasattr(result,'x'): 
        break
    result = opt.differential_evolution(objFunc, bounds, ncore=32, 
                                           prev_result=previous_result, 
                                           disp=True, maxtime=60*10)
    with open('result.thin.2sig.sext.oct','wb') as fp:
        pickle.dump(result,fp)
        time.sleep(0.1)

differential_evolution step 0: f(x)= 4.17981e-11
differential_evolution step 0: f(x)= 4.17981e-11
differential_evolution step 1: f(x)= 4.17981e-11
differential_evolution step 2: f(x)= 4.17981e-11
differential_evolution step 3: f(x)= 4.17981e-11
differential_evolution step 4: f(x)= 4.17981e-11
differential_evolution step 5: f(x)= 4.17981e-11
differential_evolution step 6: f(x)= 4.17981e-11
differential_evolution step 7: f(x)= 4.17981e-11
differential_evolution step 8: f(x)= 4.16028e-11
differential_evolution step 9: f(x)= 4.16028e-11
differential_evolution step 10: f(x)= 4.16028e-11
differential_evolution step 11: f(x)= 4.16028e-11
differential_evolution step 12: f(x)= 4.16028e-11
differential_evolution step 13: f(x)= 4.16028e-11
differential_evolution step 14: f(x)= 4.16028e-11
differential_evolution step 15: f(x)= 4.16028e-11
differential_evolution step 16: f(x)= 4.16028e-11
differential_evolution step 17: f(x)= 4.16028e-11
differential_evolution step 18: f(x)= 4.16028e-11
differentia

differential_evolution step 15: f(x)= 2.88528e-11
differential_evolution step 16: f(x)= 2.88528e-11
differential_evolution step 17: f(x)= 2.88528e-11
differential_evolution step 18: f(x)= 2.88528e-11
differential_evolution step 19: f(x)= 2.88528e-11
differential_evolution step 20: f(x)= 2.88528e-11
differential_evolution step 21: f(x)= 2.88528e-11
differential_evolution step 22: f(x)= 2.88528e-11
differential_evolution step 23: f(x)= 2.88528e-11
differential_evolution step 0: f(x)= 2.88528e-11
differential_evolution step 0: f(x)= 2.88528e-11
differential_evolution step 1: f(x)= 2.88528e-11
differential_evolution step 2: f(x)= 2.88528e-11
differential_evolution step 3: f(x)= 2.88528e-11
differential_evolution step 4: f(x)= 2.88528e-11
differential_evolution step 5: f(x)= 2.88528e-11
differential_evolution step 6: f(x)= 2.88528e-11
differential_evolution step 7: f(x)= 2.88528e-11
differential_evolution step 8: f(x)= 2.88528e-11
differential_evolution step 9: f(x)= 2.88528e-11
differentia

differential_evolution step 6: f(x)= 2.70116e-11
differential_evolution step 7: f(x)= 2.70116e-11
differential_evolution step 8: f(x)= 2.70116e-11
differential_evolution step 9: f(x)= 2.70116e-11
differential_evolution step 10: f(x)= 2.70116e-11
differential_evolution step 11: f(x)= 2.70116e-11
differential_evolution step 12: f(x)= 2.70116e-11
differential_evolution step 13: f(x)= 2.70116e-11
differential_evolution step 14: f(x)= 2.70116e-11
differential_evolution step 15: f(x)= 2.70116e-11
differential_evolution step 16: f(x)= 2.70116e-11
differential_evolution step 17: f(x)= 2.70116e-11
differential_evolution step 18: f(x)= 2.70116e-11
differential_evolution step 19: f(x)= 2.70116e-11
differential_evolution step 20: f(x)= 2.70116e-11
differential_evolution step 21: f(x)= 2.70116e-11
differential_evolution step 22: f(x)= 2.70116e-11
differential_evolution step 23: f(x)= 2.70116e-11
differential_evolution step 0: f(x)= 2.70116e-11
differential_evolution step 0: f(x)= 2.70116e-11
differ

differential_evolution step 22: f(x)= 2.34923e-11
differential_evolution step 23: f(x)= 2.34923e-11
differential_evolution step 0: f(x)= 2.34923e-11
differential_evolution step 0: f(x)= 2.34923e-11
differential_evolution step 1: f(x)= 2.34923e-11
differential_evolution step 2: f(x)= 2.34923e-11
differential_evolution step 3: f(x)= 2.34923e-11
differential_evolution step 4: f(x)= 2.34923e-11
differential_evolution step 5: f(x)= 2.34923e-11
differential_evolution step 6: f(x)= 2.34923e-11
differential_evolution step 7: f(x)= 2.34923e-11
differential_evolution step 8: f(x)= 2.34923e-11
differential_evolution step 9: f(x)= 2.34923e-11
differential_evolution step 10: f(x)= 2.34923e-11
differential_evolution step 11: f(x)= 2.34923e-11
differential_evolution step 12: f(x)= 2.34923e-11
differential_evolution step 13: f(x)= 2.34923e-11
differential_evolution step 14: f(x)= 2.34923e-11
differential_evolution step 15: f(x)= 2.34923e-11
differential_evolution step 16: f(x)= 2.34923e-11
differentia

In [ ]:
# result.x = copy(result.population[0]-0.5)
# result.x[:nThin] = result.x[:nThin]*200
# result.x

In [32]:
result

     fun: 9.079539774730545e-12
 message: 'Optimization terminated successfully.'
     nit: 18
 success: True
       x: array([ -31.00299848,   -4.09397977,    5.10680353,   -3.7397568 ,
          6.73863865,   -8.25961454,  -30.85943659,   13.69605324,
          1.92247865,   16.07983921,  -12.07710141,  -26.57980804,
        -10.44255265,  -10.09065893,   33.68369676,  -34.62787716,
        -41.58941562,   -1.10227422,  483.02648308,  306.29666258,
       -349.0077146 ,   27.79818619,   -8.09121491,   19.97538273,
        290.71988547, -386.4007979 , -227.67167027, -470.46805636,
        178.23790314,  468.49375363, -183.170586  ,  336.06589167,
        113.56853006,  482.83624056,  215.24236131, -379.98983798])